# 目的
gemmaを使う、外部データ２を追加する

ref.
- https://www.kaggle.com/datasets/abdullahmeda/lmsys-additional-33k-labelled-conversations?select=lmsys-33k-deduplicated.csv
- https://www.kaggle.com/code/emiz6413/training-gemma-2-9b-4-bit-qlora-fine-tuning

In [1]:
# path setting
EXP_NAME = "e071-add-ex-data2-ep2"
MODEL_NAME = "unsloth/gemma-2-9b-it-bnb-4bit"
COMPETITION_NAME = "lmsys"

DATA_PATH = "data"
DATASET_NAME = f"{EXP_NAME}-{MODEL_NAME.split('/')[-1]}"
MODEL_OUTPUT_PATH = f"trained_models/{EXP_NAME}"

# experiment parameter
DEBUG = False
TRAINING = True
UPLOAD_DATA_TO_S3 = True
UPLOAD_DATA_TO_KAGGLE = True
REMOVE_LOCAL_FILE = False
WANDB = True
# USE_FOLD = 2
USE_DATA_RATE = 1.0
# VALID_DATA_SIZE = 3000

# model parameter
TRAINING_MAX_LENGTH = 2048 # 512
# INFERENCE_MAX_LENGTH = 1536
SEED = 42
EPOCH = 2
LR = 2e-05 # 2e-04
TRAIN_BS = 4 # 16
GRAD_ACC_STEP= 128 // TRAIN_BS # 仮想的なバッチサイズはTRAIN_BS * GRAD_ACC_STEPとなる
EVAL_BS = 4 # 16
NUM_LABELS = 3

FREEZE_LAYERS = (
    0  # there're 42 layers in total, we don't add adapters to the first 16 layers
)

# rola parameter
LORA_R = 16
LORA_ALPHA = LORA_R * 2
LORA_DROPOUT = 0.05
LORA_BIAS = "none"

RESUME_FROM_CHECKPOINT= True # 途中から再開する場合はTrueにする
# TRAINED_MODEL_PATH = "lmsys/trained_models/e006-use-concat"

In [2]:
!nvidia-smi

In [3]:
!python --version

In [4]:
def resolve_path(base_path: str) -> str:
    import os

    cwd = os.getcwd()
    print(cwd)
    if cwd == f"/notebooks":
        print("Jupyter Kernel By VSCode!")
        return "kernel", f"/notebooks/{COMPETITION_NAME}/{base_path}"
    elif cwd == f"/notebooks/{COMPETITION_NAME}":
        print("nohup!")
        return base_path
    elif cwd == f"/notebooks/{COMPETITION_NAME}/{COMPETITION_NAME}/exp":
        print("Jupyter Lab!")
        return "nohup", f"../../{base_path}"
    elif cwd == f"/content":
        print("Google Colab!")
        return "colab", f"/content/drive/MyDrive/Kaggle/{COMPETITION_NAME}/{base_path}"
    elif cwd.startswith("/home/shinichiro.saito"):
        print("GCP!")
        return "GCP", f"/home/shinichiro.saito/{COMPETITION_NAME}/{base_path}"
    else:
        raise Exception("Unknown environment")


ENV_NAME, DATA_PATH = resolve_path(DATA_PATH)
print(DATA_PATH)
_, MODEL_OUTPUT_PATH = resolve_path(MODEL_OUTPUT_PATH)
print(MODEL_OUTPUT_PATH)

In [5]:
def validate_dataset_name(dataset_name: str) -> None:
    if len(dataset_name) < 6 or len(dataset_name) > 50:
        raise Exception(
            f"データセットの文字列は6~50文字にしてください。現在{len(DATASET_NAME)}文字"
        )
    if "_" in dataset_name:
        raise Exception("datasetの名称に_の使用は禁止です")


validate_dataset_name(DATASET_NAME)

# install

In [6]:
if ENV_NAME != "GCP":
    %pip install -qq polars==1.0.0
    %pip install -qq transformers==4.42.3
    %pip install -qq datasets==2.20.0
    %pip install -qq evaluate==0.4.2
    %pip install -qq seqeval==1.2.2
    %pip install -qq accelerate==0.32.0
    %pip install -qq python-dotenv==1.0.1
    %pip install -qq wandb==0.17.4
    %pip install -qq bitsandbytes==0.43.1
    %pip install -qq accelerate==0.32.0
    %pip install -qq peft==0.11.1

    # formatter
    %pip install -qq black isort

    %pip install -qq kaggle

# import

In [7]:
import os
import random
import ast
import json

import polars as pl
import numpy as np
import torch
import torch.nn as nn
from torch.nn import BCEWithLogitsLoss, CrossEntropyLoss, LayerNorm, MSELoss
import wandb
from datasets import (
    Dataset,
    DatasetDict,
    Value,
    concatenate_datasets,
    load_dataset,
    ClassLabel,
)
from tokenizers import AddedToken
from tqdm.auto import tqdm
from scipy.special import softmax
from sklearn.metrics import log_loss
from transformers import (
    AutoConfig,
    AutoModel,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    DataCollatorWithPadding,
    Trainer,
    TrainingArguments,
    BitsAndBytesConfig,
    Gemma2ForSequenceClassification,
    GemmaTokenizerFast,
    Gemma2Config,
    PreTrainedTokenizerBase,
    EvalPrediction,
    Trainer,
    DataCollatorWithPadding,
    TrainingArguments,
)

from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, TaskType

from sklearn.metrics import log_loss, accuracy_score

In [8]:
os.environ["TOKENIZERS_PARALLELISM"] = "true"
NUM_PROC = os.cpu_count()

In [9]:
import transformers
import datasets
import evaluate
import bitsandbytes
import accelerate
import peft

assert transformers.__version__ == "4.42.3"
assert datasets.__version__ == "2.20.0"
assert evaluate.__version__ == "0.4.2"
assert bitsandbytes.__version__ == "0.43.1"
assert accelerate.__version__ == "0.32.0"
assert peft.__version__ == "0.11.1"

In [10]:
# Seed the same seed to all
def seed_everything(seed: int):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True


seed_everything(SEED)

In [11]:
from dotenv import load_dotenv

load_dotenv(f"{DATA_PATH}/.env")

True

# Wandb

In [12]:
if WANDB:
    wandb.login(key=os.environ["WANDB_API_KEY"])
    wandb.init(project=COMPETITION_NAME, name=EXP_NAME)
    REPORT_TO = "wandb"
else:
    REPORT_TO = "none"

REPORT_TO

'wandb'

# Data Import & Preprocess

In [13]:
# with open(f"{DATA_PATH}/label_stratified_fold.json") as f:
#     label_stratified_fold = json.load(f)

In [14]:
train = (
    pl.read_csv(f"{DATA_PATH}/train.csv")
    .with_columns(pl.col("id").cast(pl.String))
    # .with_columns(
    #     pl.col("prompt").str.json_decode(),
    #     pl.col("response_a").str.json_decode(),
    #     pl.col("response_b").str.json_decode(),
    # )
    # .with_columns(  # 長さの情報を追加する
    #     pl.col("prompt")
    #     .map_elements(lambda x: len(x), return_dtype=pl.Int64)
    #     .alias("len_prompt"),
    #     pl.col("response_a")
    #     .map_elements(lambda x: len(x), return_dtype=pl.Int64)
    #     .alias("len_response_a"),
    #     pl.col("response_b")
    #     .map_elements(lambda x: len(x), return_dtype=pl.Int64)
    #     .alias("len_response_b"),
    # )
    # .with_columns(  # 最後のレスポンスのみを取得する
    #     pl.col("prompt")
    #     .map_elements(lambda x: x[-1], return_dtype=pl.String)
    #     .alias("last_prompt"),
    #     pl.col("response_a")
    #     .map_elements(lambda x: x[-1], return_dtype=pl.String)
    #     .alias("last_response_a"),
    #     pl.col("response_b")
    #     .map_elements(lambda x: x[-1], return_dtype=pl.String)
    #     .alias("last_response_b"),
    # )
    # .with_columns(  # 最後のレスポンスがNoneの場合を空文字にする、約60件程度
    #     pl.col("last_response_a").fill_null(""),
    #     pl.col("last_response_b").fill_null(""),
    # )
    # .with_columns(  # labelを付与する
    #     pl.when(pl.col("winner_model_a") == 1)
    #     .then(0)
    #     .when(pl.col("winner_model_b") == 1)
    #     .then(1)
    #     .when(pl.col("winner_tie") == 1)
    #     .then(2)
    #     .alias("label"),
    # )
    # .select(  # 元のprompt, responseを削除する
    #     pl.exclude(["prompt", "response_a", "response_b"])
    # )
    # .with_columns(  # foldを追加する
    #     pl.col("id").replace(label_stratified_fold).alias("fold")
    # )
)

In [15]:
# https://www.kaggle.com/code/takamichitoda/lmsys-convert-ultrafeedback-to-competition/output
# extra_data = pl.read_csv(f"{DATA_PATH}/ultrachat_s42_a0.5.csv")

# https://www.kaggle.com/datasets/abdullahmeda/lmsys-additional-33k-labelled-conversations?select=lmsys-33k-deduplicated.csv
extra_data = pl.read_csv(f"{DATA_PATH}/lmsys-33k-deduplicated.csv")

In [16]:
print(extra_data["winner_model_a"].value_counts())
print(extra_data["winner_model_b"].value_counts())
print(extra_data["winner_tie"].value_counts())

In [17]:
train = pl.concat([train, extra_data], how="vertical")

In [18]:
if DEBUG:
    train = train.sample(fraction=1.0, shuffle=True).head(100)

In [19]:
train_dataset = Dataset.from_polars(train)

In [20]:
train_dataset

Dataset({
    features: ['id', 'model_a', 'model_b', 'prompt', 'response_a', 'response_b', 'winner_model_a', 'winner_model_b', 'winner_tie'],
    num_rows: 78664
})

In [21]:
# 計算を早くするために、データを減らす
if not DEBUG:
    train_dataset = train_dataset.shuffle().select(
        range(
            int(len(train_dataset) * USE_DATA_RATE)
        )
    )

# Model

In [22]:
lora_config = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA, # low-rankマトリクスのスケーリングファクター
    # only target self-attention
    # target_modules=["q_proj", "k_proj", "v_proj"],
    target_modules=[ # Linear層を全て含める
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj"
    ],
    layers_to_transform=[i for i in range(42) if i >= FREEZE_LAYERS],
    lora_dropout=LORA_DROPOUT, # LoRAレイヤーのドロップアウト確率
    bias=LORA_BIAS,
    task_type=TaskType.SEQ_CLS,
)

In [23]:
# tokenizer = GemmaTokenizerFast.from_pretrained(MODEL_NAME)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.add_eos_token = True  # We'll add <eos> at the end
tokenizer.padding_side = "right"
# tokenizer.add_special_tokens({"additional_special_tokens": ["[SEP]"]})

In [24]:
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=NUM_LABELS,
    torch_dtype=torch.float16,
    device_map="auto",
    cache_dir="./model_cache"
)
model.config.use_cache = False
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)
# model.resize_token_embeddings(len(tokenizer), pad_to_multiple_of=16)
print(model)
print(model.print_trainable_parameters())

In [25]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, pad_to_multiple_of=16)

# Tokenize

In [26]:
def cut_most_length_text(prompt: str, response_a: str, response_b: str, cut_num: int):

    prompt_len = len(prompt.split(" "))
    response_a_len = len(response_a.split(" "))
    response_b_len = len(response_b.split(" "))
    
    if (prompt_len >= response_a_len) & (prompt_len >= response_b_len):
        prompt = " ".join(prompt.split(" ")[:-cut_num])
    elif (response_a_len >= prompt_len) & (response_a_len >= response_b_len):
        response_a = " ".join(response_a.split(" ")[:-cut_num])
    elif (response_b_len >= prompt_len) & (response_b_len >= response_a_len):
        response_b = " ".join(response_b.split(" ")[:-cut_num])
    else:
        raise Exception
    
    return prompt, response_a, response_b

def tokenize(example):
    cut_len = TRAINING_MAX_LENGTH - 16 # bos, '\n\n', '<', 'prompt', '>:',  '<', 'response', '_', 'a', '>:', '<',  'response',  '_',  'b',  '>:', eos
    
    # 要素が　null　の場合を""に置換する
    # listにする
    prompt = " ".join(eval(example["prompt"], {"null": ""}))
    response_a = " ".join(eval(example["response_a"], {"null": ""}))
    response_b = " ".join(eval(example["response_b"], {"null": ""}))
    
    prompt = "<prompt>: " + prompt
    response_a = "\n\n<response_a>: " + response_a
    response_b = "\n\n<response_b>: " + response_b
    
    texts = prompt + response_a + response_b
    
    # 収まるまで、最も長いテキストを1単語ずつ減らす
    # 遅い、5000件で1分ぐらいかかる
    while len(texts.split(" ")) > cut_len:
        prompt, response_a, response_b = cut_most_length_text(prompt, response_a, response_b, 1)
        texts = prompt + response_a + response_b
    
    tokenized = tokenizer(texts, max_length=TRAINING_MAX_LENGTH, truncation=True)
    
    if example["winner_model_a"]:
        label = 0
    elif example["winner_model_b"]:
        label = 1
    else:
        label = 2
        
    return {**tokenized, "labels": label}

In [27]:
train_dataset = train_dataset.map(
    tokenize,
    batched=False
)

Map:   0%|          | 0/78664 [00:00<?, ? examples/s]

In [28]:
print(train_dataset)

In [29]:
# def tokenize(examples, max_token_length: int):
#     separator = " [SEP] "

#     joined_text = (
#         examples["last_prompt"]
#         + separator
#         + examples["last_response_a"]
#         + separator
#         + examples["last_response_b"]
#     )

#     return tokenizer(
#         joined_text,
#         max_length=max_token_length,
#         truncation=True,
#         padding="max_length",
#     )


# train_dataset = train_dataset.map(
#     tokenize,
#     batched=False,
#     fn_kwargs={"max_token_length": TRAINING_MAX_LENGTH},
#     num_proc=NUM_PROC,
# )

# Train Test Split

In [30]:
# filtered_train = train_dataset.filter(lambda x: x["fold"] != USE_FOLD, num_proc=NUM_PROC)
# filtered_valid = train_dataset.filter(lambda x: x["fold"] == USE_FOLD, num_proc=NUM_PROC)
# filtered_valid = filtered_valid.select(range(min(VALID_DATA_SIZE, len(filtered_valid))))

# train_valid_dataset = DatasetDict(
#     {
#         "train": filtered_train,
#         "valid": filtered_valid,
#     }
# )

# del filtered_train, filtered_valid

In [31]:
print(train_dataset)

In [32]:
# def compute_metrics(eval_pred):
#     predictions, labels = eval_pred
#     preds_prob = softmax(predictions, axis=-1)
#     return {"log_loss": log_loss(labels, preds_prob)}


def compute_metrics(eval_pred) -> dict:
    preds, labels = eval_pred
    preds_prob = softmax(preds, axis=-1)
    return {
        "log_loss": log_loss(y_true=labels, y_pred=preds_prob),
        "acc": accuracy_score(y_true=labels, y_pred=preds.argmax(-1)),
    }

In [33]:
# スケジューラの設定
training_args = TrainingArguments(
    output_dir=MODEL_OUTPUT_PATH,
    overwrite_output_dir=True,
    learning_rate=LR,
    per_device_train_batch_size=TRAIN_BS,
    gradient_accumulation_steps=GRAD_ACC_STEP,
    eval_accumulation_steps=GRAD_ACC_STEP,
    # per_device_eval_batch_size=EVAL_BS,
    num_train_epochs=EPOCH,
    weight_decay=0.01,
    do_eval=False,
    # eval_strategy="steps",
    eval_strategy="no",
    # eval_steps=0.1,
    save_strategy="steps",
    save_steps=0.1,
    save_total_limit=10,
    logging_steps=2,
    seed=SEED,
    # metric_for_best_model="eval_loss",
    warmup_ratio=0.1,
    lr_scheduler_type="cosine_with_restarts", # "linear", # "constant_with_warmup",
    report_to=REPORT_TO,
    run_name=EXP_NAME,
    # load_best_model_at_end=True,
    fp16=True,
    # fp16_full_eval=True,
    gradient_checkpointing=True,
    optim="adamw_8bit",
)

trainer = Trainer(
    model=model,
    args=training_args,
    # train_dataset=train_valid_dataset["train"],
    train_dataset=train_dataset,
    # train_dataset=ConcatDataset(train_valid_dataset["train"]),
    # eval_dataset=train_valid_dataset["valid"],
    # eval_dataset=ConcatDataset(train_valid_dataset["valid"]),
    tokenizer=tokenizer,
    data_collator=data_collator,
    # compute_metrics=compute_metrics,
)

In [34]:
if TRAINING:
    # モデルの学習
    trainer.train(
        resume_from_checkpoint = RESUME_FROM_CHECKPOINT if RESUME_FROM_CHECKPOINT else None
    )
    # ログの保存に利用したストレージを削除
    # os.system(f"rm -rf {MODEL_OUTPUT_PATH}/checkpoint-*")
    # モデルの保存
    trainer.save_model(MODEL_OUTPUT_PATH)
else:
    # TRAINED_MODEL_PATHを用いて、学習済のモデルを読み込む
    model = AutoModelForSequenceClassification.from_pretrained(
        TRAINED_MODEL_PATH,
        num_labels=NUM_LABELS,
    )
    # model = CustomDebertaSequenceClassification.from_pretrained(MODEL_NAME)

    args = TrainingArguments(
        ".",
        per_device_eval_batch_size=4,
        report_to="none",
        fp16=True,
    )

    trainer = Trainer(
        model=model,
        args=args,
        data_collator=data_collator,
        tokenizer=tokenizer,
    )

Step,Training Loss
616,0.737700
618,0.721400
620,0.753700
622,0.789200
624,0.741900
626,0.738900
628,0.806300
630,0.733200
632,0.768700
634,0.788800


```
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
TODO: この　Warningが問題ないのかを調べる

```

# valid_datasetの作成・保存

In [35]:
# # TRAININGをINFERRENCEでMAX_TOKENを変えるために、validを作り直す
# valid_dataset = train_dataset.filter(
#     lambda example: example["id"] in train_valid_dataset["valid"]["id"],
#     num_proc=NUM_PROC,
# )

# valid_dataset = valid_dataset.map(
#     CustomTokenizer(tokenizer, max_length=INFERENCE_MAX_LENGTH),
#     batched=True,
#     num_proc=NUM_PROC,
# )

# # valid_dataset = valid_dataset.map(
# #     tokenize,
# #     batched=False,
# #     fn_kwargs={"max_token_length": INFERENCE_MAX_LENGTH},
# #     num_proc=NUM_PROC,
# # )

# # valid_dataset = valid_dataset.map(
# #     tokenize,
# #     batched=False,
# #     fn_kwargs={
# #         "suffix": "a",
# #         "max_token_length": INFERENCE_MAX_LENGTH
# #     },
# #     num_proc=NUM_PROC,
# # ).map(
# #     tokenize,
# #     batched=False,
# #     fn_kwargs={
# #         "suffix": "b",
# #         "max_token_length": INFERENCE_MAX_LENGTH
# #     },
# #     num_proc=NUM_PROC,
# # )


# def add_valid_pred(example, idx, valid_pred):
#     example["valid_pred"] = valid_pred[idx]
#     return example


# valid_dataset = train_valid_dataset["valid"]

# valid_pred = softmax(trainer.predict(valid_dataset).predictions, axis=-1)
# # valid_pred = softmax(trainer.predict(ConcatDataset(valid_dataset)).predictions, axis=-1)

# np.save(f"{MODEL_OUTPUT_PATH}/valid_prediction.npy", valid_pred)

# valid_dataset = valid_dataset.map(
#     add_valid_pred, with_indices=True, fn_kwargs={"valid_pred": valid_pred}
# )

# valid_dataset.save_to_disk(f"{MODEL_OUTPUT_PATH}/valid_dataset")

# CVの計算

In [36]:
# cv_score = log_loss(valid_dataset["labels"], valid_pred)
# print(f"CV Score: {cv_score}")

In [37]:
# # output_textを保存
# with open(f"{MODEL_OUTPUT_PATH}/cv_score.txt", "w") as f:
#     f.write(str(cv_score))

# AWSへのアップロード

In [38]:
# S3へのアップロード
# TODO: colabでは動かないため直す
if not DEBUG and UPLOAD_DATA_TO_S3:
    # uninstall
    !sudo rm /usr/bin/aws
    !sudo rm /usr/bin/aws_completer
    !sudo rm -rf /usr/local/aws-cli

    # install
    !curl "https://awscli.amazonaws.com/awscli-exe-linux-x86_64.zip" -o "awscliv2.zip"
    !unzip -o -qq awscliv2.zip
    !sudo ./aws/install --update

    # upload
    output_name = MODEL_OUTPUT_PATH.split("/")[-1]
    os.system(
        f"aws s3 cp --recursive {MODEL_OUTPUT_PATH} s3://{COMPETITION_NAME}/trained_model/{output_name}"
    )

In [39]:
# ダウンロード（参考）
# !sudo rm /usr/bin/aws
# !sudo rm /usr/bin/aws_completer
# !sudo rm -rf /usr/local/aws-cli

# !curl "https://awscli.amazonaws.com/awscli-exe-linux-x86_64.zip" -o "awscliv2.zip"
# !unzip -o -qq awscliv2.zip
# !sudo ./aws/install --update

# !aws s3 cp --recursive s3://automated-essay-scoring/trained_model/e005-regression /notebooks/automated_essay_scoring/trained_models/e005-regression

# Kaggle Datasetへのupload

In [40]:
import os

os.system("mkdir -p ~/.kaggle/")
os.system(f"cp /{DATA_PATH}/kaggle.json ~/.kaggle/")
os.system("chmod 600 ~/.kaggle/kaggle.json")

0

In [41]:
if not DEBUG and UPLOAD_DATA_TO_KAGGLE:
    import os
    import json

    from kaggle.api.kaggle_api_extended import KaggleApi

    def dataset_create_new(dataset_name: str, upload_dir: str):
        # if "_" in dataset_name:
        #     raise ValueError("datasetの名称に_の使用は禁止です")
        dataset_metadata = {}
        dataset_metadata["id"] = f"sinchir0/{dataset_name}"
        dataset_metadata["licenses"] = [{"name": "CC0-1.0"}]
        dataset_metadata["title"] = dataset_name
        with open(os.path.join(upload_dir, "dataset-metadata.json"), "w") as f:
            json.dump(dataset_metadata, f, indent=4)
        api = KaggleApi()
        api.authenticate()
        api.dataset_create_new(folder=upload_dir, convert_to_csv=False, dir_mode="tar")

    print(f"Create Dataset name:{DATASET_NAME}, output_dir:{MODEL_OUTPUT_PATH}")
    dataset_create_new(dataset_name=DATASET_NAME, upload_dir=MODEL_OUTPUT_PATH)

# ローカルからのデータの削除

In [42]:
# if not DEBUG and REMOVE_LOCAL_FILE:
#     # ローカルからは削除
#     os.system(f"rm -rf {MODEL_OUTPUT_PATH}")

In [43]:
if WANDB:
    wandb.finish()

In [44]:
print("finish Notebook!")